In [1]:
from bs4 import BeautifulSoup
import csv

In [2]:
file_path = "data/Resume.csv"
rows = []
with open(file_path, 'r', encoding='utf-8') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)
     
    # extracting field names through first row
    fields = next(csvreader)
    
    for row in csvreader:
        rows.append(row)

In [3]:
def extract_content(sec):
    ct = sec.find_all("div", {"class": "paragraph"})
    rt = []
    for p in ct:
        for s in p.stripped_strings:
            rt.append(s)
    return rt

In [4]:
from collections import defaultdict
freq = defaultdict(int)
cb = defaultdict(int)
data = {}

idx = 0
for row in rows:
    data["row"+str(idx)] = []
    soup = BeautifulSoup(row[2], 'html.parser')
    comb = []
    a = soup.find_all("div", {"class": "section"})
    for s in a:
        b = s.find("div", {"class": "sectiontitle"})
        if b:
            dt = {}
            title = b.text.strip()
            freq[title] += 1
            comb.append(title)
            dt["id"] = row[0]
            dt["title"] = title
            dt["content"] = extract_content(s)
            dt["category"] = row[3]
            data["row"+str(idx)].append(dt)
    comb.sort()
    k = "_".join(comb)
    cb[k] += 1
    idx += 1

In [5]:
print(data["row2000"])

[{'id': '12212468', 'title': 'Summary', 'content': ["Highly motivated Civil Engineer with 16 years of experience working for one of North America's largest and most respected construction and engineering organizations with revenues of more than $11 billion and consistently ranking among the top five contractors by Engineering News-Record.", 'My experience with Kiewit has prepared me to successfully plan and execute projects while promoting safety, quality, schedule, and cost performance. My career experience ranges from construction design coordinator on a multi billion dollar design build project, to on-site project engineering and project management for self-performing heavy civil construction work ranging from $545 million to $3.1 million dollars. Excellent communicator skilled in construction means and methods, cost controls, scheduling, claims analysis and contract administration.', 'As the Field Engineer and Project Controls Manager on one of the largest design-build project in M

In [6]:
s = BeautifulSoup(rows[2000][2], 'html.parser')
print(s.prettify())

<div class="fontsize fontface vmargins hmargins linespacing pagesize" id="document">
 <div class="section firstsection" id="SECTION_NAME585766938" style="
      padding-top:0px;
    ">
  <div class="paragraph PARAGRAPH_NAME firstparagraph" id="PARAGRAPH_585766938_1_381272156" style="
      padding-top:0px;
    ">
   <div class="name thinbottomborder" itemprop="name">
    <span class="field fName" id="585766938FNAM1">
    </span>
    <span>
    </span>
    <span class="field" id="585766938LNAM1">
     ASSISTANT PROJECT MANAGER/DESIGN BUILD CONSTRUCTION COORDINATOR
    </span>
   </div>
  </div>
 </div>
 <div class="section" id="SECTION_SUMM585766940" style="
      padding-top:0px;
    ">
  <div class="heading">
   <div class="sectiontitle" id="SECTNAME_SUMM585766940">
    Summary
   </div>
  </div>
  <div class="paragraph firstparagraph" id="PARAGRAPH_585766940_1_381270288" style="
      padding-top:0px;
    ">
   <div class="field singlecolumn noPind" id="585766940FRFM1">
    <p align=

In [7]:
print(s.get_text())

         ASSISTANT PROJECT MANAGER/DESIGN BUILD CONSTRUCTION COORDINATOR       Summary     Highly motivated Civil Engineer with 16 years of experience working for one of North America's largest and most respected construction and engineering organizations with revenues of more than $11 billion and consistently ranking among the top five contractors by Engineering News-Record.   My experience with Kiewit has prepared me to successfully plan and execute projects while promoting safety, quality, schedule, and cost performance. My career experience ranges from construction design coordinator on a multi billion dollar design build project, to on-site project engineering and project management for self-performing heavy civil construction work ranging from $545 million to $3.1 million dollars. Excellent communicator skilled in construction means and methods, cost controls, scheduling, claims analysis and contract administration.   As the Field Engineer and Project Controls Manager on one of t

In [8]:
fq = list(freq.items())
fq.sort(key= lambda x: x[1], reverse=True)
print(fq)

[('Skills', 2700), ('Education', 1907), ('Experience', 1714), ('Summary', 1344), ('Highlights', 876), ('Accomplishments', 761), ('Education and Training', 508), ('Additional Information', 462), ('Professional Summary', 460), ('Professional Experience', 401), ('Work History', 351), ('Certifications', 281), ('Languages', 238), ('Interests', 234), ('Professional Affiliations', 211), ('Core Qualifications', 208), ('Skill Highlights', 187), ('Affiliations', 156), ('Executive Profile', 131), ('Personal Information', 130), ('Work Experience', 128), ('Qualifications', 100), ('Core Accomplishments', 93), ('Career Overview', 85), ('Activities and Honors', 58), ('Professional Profile', 51), ('Career Focus', 48), ('Profile', 47), ('Technical Skills', 44), ('Publications', 43), ('Presentations', 39), ('Core Strengths', 35), ('Executive Summary', 34), ('Summary of Skills', 34), ('Educational Background', 33), ('Relevant Experience', 29), ('Military Experience', 27), ('Objective', 23), ('Computer Ski

In [9]:
c = list(cb.items())
c.sort(key= lambda x: x[1], reverse=True)
print(c)

[('Accomplishments_Education_Experience_Highlights_Skills_Summary', 195), ('Education_Experience_Highlights_Skills_Summary', 175), ('Education_Experience_Skills_Summary', 74), ('Education and Training_Experience_Skills_Summary', 55), ('Education and Training_Experience_Skills_Skills_Summary', 53), ('Education_Professional Summary_Skills_Skills_Work History', 51), ('Core Qualifications_Education_Experience_Professional Summary_Skills', 34), ('Core Accomplishments_Education_Executive Profile_Professional Experience_Skill Highlights_Skills', 29), ('Education_Experience_Highlights_Languages_Skills_Summary', 27), ('Accomplishments_Education_Experience_Skills_Summary', 25), ('Accomplishments_Education_Experience_Highlights_Languages_Skills_Summary', 20), ('Accomplishments_Education_Experience_Highlights_Professional Affiliations_Skills_Summary', 18), ('Accomplishments_Certifications_Education_Experience_Highlights_Skills_Summary', 18), ('Core Qualifications_Education_Experience_Professional 

In [10]:
content_file = "output/resume_content.txt"
with open(content_file, "w", encoding='utf-8') as f:
    for idx in range(2484):
        line = data["row"+str(idx)]
        print(line, file=f)

In [11]:
freq_file = "output/title_freq.txt"
with open(freq_file, "w", encoding='utf-8') as f:
    for title, count in sorted(freq.items(),
                             key=lambda x: x[1], reverse=True):
        line = '{}_{}'.format(title, count)
        print(line, file=f)